## Downloading the next two days of data for weather.gov

In [56]:
import sys
from datetime import datetime

import requests
import pytz

In [72]:
c_park = {"latitude":40.7823,
         "longitude":-87.7421}

In [73]:
base_url = "https://api.weather.gov/points/{latitude},{longitude}"

In [74]:
def format_url(base_url, latitude, longitude):
    url = base_url.format(latitude=latitude, longitude=longitude)
    return f"{url}"

In [75]:
url = format_url(base_url, **c_park)

In [76]:
url

'https://api.weather.gov/points/40.7823,-87.7421'

In [77]:
d = requests.get(url)

In [78]:
d

<Response [200]>

In [79]:
d.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/40.7823,-87.7420999',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-87.7420999, 40.7823]},
 'properties': {'@id': 'https://api.weather

In [50]:
hourly_url = d.json()["properties"]["forecastHourly"]

In [51]:
d = requests.get(hourly_url)

In [52]:
d

<Response [200]>

In [58]:
d.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.7673534, 40.7986966],
    [-87.76889800000001, 40.7767939],
    [-87.7399666, 40.7756218],
    [-87.7384159, 40.7975245],
    [-87.7673534, 40.7986966]]]},
 'properties': {'updated': '2021-10-10T15:15:41+00:00',
  'units': 'us',
  'forecastGenerator': 'HourlyForecastGenerator',
  'generatedAt': '2021-10-10T15:38:23+00:00',
  'updateTime': '2021-10-10T15:15:41+00:00',
  'validTimes': '2021-10-10T09:00:00+00:00/P7DT16H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 185.0136},
  'periods': [{'number': 1,
    'name': '',
    'startTime': '2021-10-10T10:00:00-05:00',
    'endTime': '2021-10-10T11:00:00-05:00',
    'isDaytime'

In [65]:
def get_url(url):
    
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"There has been an error: {err}")
        return err
    return response

In [69]:
def wrapper(location_json):
    url = format_url(base_url, **location_json)
    
    r = get_url(url)
    
    forecast_url = r.json()["properties"]["forecastHourly"]
    
    r = get_url(forecast_url)
    
    return r
    
j = wrapper(c_park)

In [71]:
j.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.7673534, 40.7986966],
    [-87.76889800000001, 40.7767939],
    [-87.7399666, 40.7756218],
    [-87.7384159, 40.7975245],
    [-87.7673534, 40.7986966]]]},
 'properties': {'updated': '2021-10-10T15:15:41+00:00',
  'units': 'us',
  'forecastGenerator': 'HourlyForecastGenerator',
  'generatedAt': '2021-10-10T15:38:23+00:00',
  'updateTime': '2021-10-10T15:15:41+00:00',
  'validTimes': '2021-10-10T09:00:00+00:00/P7DT16H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 185.0136},
  'periods': [{'number': 1,
    'name': '',
    'startTime': '2021-10-10T10:00:00-05:00',
    'endTime': '2021-10-10T11:00:00-05:00',
    'isDaytime'